In [17]:
## RECSYS CHALLENGE 2020 ##
# Evaluation or scoring?
eval = True

In [18]:
# Loading from CSV files...
from RecLib.DataLoad import *
UCM_age, ICM_subclass, ICM_asset, ICM_price, UCM_region, target_users, URM = dataLoad()

In [19]:
import scipy.sparse as sps
URM = URM.tocsr()

In [20]:
# Split dataset (train % of .9999 gives similar performance on test set and competition set)
from Notebooks_utils.data_splitter import train_test_holdout
URM_train, URM_test = train_test_holdout(URM, train_perc = 0.8)


if not eval:
    URM_train = URM
else:
    from Base.Evaluation.Evaluator import EvaluatorHoldout
    from RecLib.Evaluate import *
    evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])

In [21]:
# Load Item & User Content matrix
ICM, UCM = contentMatrixLoad(URM_train, ICM_subclass, ICM_price, ICM_asset, UCM_age, UCM_region)

In [22]:
if True:
    from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender
    SLIMElasticNet = SLIMElasticNetRecommender(URM_train)
    MAP_LIST = []
    tklist = [30, 50, 60]
    penalties = [1]
    alphas = [0.2, 0.4, 0.7, 1]
    tol = [1e-3, 1e-4, 1e-5]
    max_iter = [100, 1000]
    positives = [False, True]
    warm_start = [True, False]
    for tk in tklist:
        for penalty in penalties:
            for alpha in alphas:
                for positivity in positives:
                    for ws in warm_start:
                        SLIMElasticNet.fit(l1_ratio=penalty, alpha = alpha, positive_only=positivity, topK = tk, warm_start = ws)
                        if eval:
                            dict_scores = (evaluator_validation.evaluateRecommender(SLIMElasticNet))[0][10]
                            MAP_LIST.append(('topK, l1_ratio, alphas :' + str(tk) + ' ' + str(penalty) + ' ' + str(alpha), dict_scores['MAP']))
    if eval:
        sortMap(MAP_LIST)

SLIMElasticNetRecommender: Processed 6284 ( 33.98% ) in 5.00 min. Items per second: 20.94
SLIMElasticNetRecommender: Processed 13964 ( 75.50% ) in 10.00 min. Items per second: 23.27
SLIMElasticNetRecommender: Processed 18495 ( 100.00% ) in 12.71 min. Items per second: 24.25
EvaluatorHoldout: Processed 20428 ( 100.00% ) in 10.01 sec. Users per second: 2040
SLIMElasticNetRecommender: Processed 6524 ( 35.27% ) in 5.00 min. Items per second: 21.74
SLIMElasticNetRecommender: Processed 13898 ( 75.14% ) in 10.00 min. Items per second: 23.15
SLIMElasticNetRecommender: Processed 18495 ( 100.00% ) in 12.76 min. Items per second: 24.16
EvaluatorHoldout: Processed 20428 ( 100.00% ) in 9.49 sec. Users per second: 2152
SLIMElasticNetRecommender: Processed 7446 ( 40.26% ) in 5.00 min. Items per second: 24.80
SLIMElasticNetRecommender: Processed 15556 ( 84.11% ) in 10.00 min. Items per second: 25.92
SLIMElasticNetRecommender: Processed 18495 ( 100.00% ) in 11.73 min. Items per second: 26.28
EvaluatorH

EvaluatorHoldout: Processed 20428 ( 100.00% ) in 9.36 sec. Users per second: 2182
SLIMElasticNetRecommender: Processed 7525 ( 40.69% ) in 5.00 min. Items per second: 25.08
SLIMElasticNetRecommender: Processed 15860 ( 85.75% ) in 10.00 min. Items per second: 26.43
SLIMElasticNetRecommender: Processed 18495 ( 100.00% ) in 11.53 min. Items per second: 26.74
EvaluatorHoldout: Processed 20428 ( 100.00% ) in 9.40 sec. Users per second: 2173
SLIMElasticNetRecommender: Processed 7544 ( 40.79% ) in 5.00 min. Items per second: 25.13
SLIMElasticNetRecommender: Processed 15883 ( 85.88% ) in 10.00 min. Items per second: 26.46
SLIMElasticNetRecommender: Processed 18495 ( 100.00% ) in 11.50 min. Items per second: 26.79
EvaluatorHoldout: Processed 20428 ( 100.00% ) in 9.41 sec. Users per second: 2171
SLIMElasticNetRecommender: Processed 7478 ( 40.43% ) in 5.00 min. Items per second: 24.92
SLIMElasticNetRecommender: Processed 15760 ( 85.21% ) in 10.00 min. Items per second: 26.26
SLIMElasticNetRecommen

In [45]:
# SLIM BPR Recommender
if True:
    from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython 
    slim_rec = SLIM_BPR_Cython(URM_train, recompile_cython=False, verbose = False)
    MAP_LIST = []
    epochsList = [300]
    batchSize = [50]
    tklist = [10]
    lrs = [1e-3]

    for epochsN in epochsList:
        for bs in batchSize:
            for tk in tklist:
                for lr in lrs:
                    do_not_display_hystory = slim_rec.fit(epochs=epochsN, batch_size=bs, sgd_mode='adagrad', learning_rate=lr, topK = tk)
                    if eval:
                        dict_scores = (evaluator_validation.evaluateRecommender(slim_rec))[0][10]
                        MAP_LIST.append(('epoch, batch, topK, lr :' + str(epochsN) + ' ' + str(bs)+ ' ' + str(tk) + ' ' + str(lr), dict_scores['MAP']))
    if eval:
        sortMap(MAP_LIST)

Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Epoch 1 of 300. Elapsed time 0.05 sec
SLIM_BPR_Recommender: Epoch 76 of 300. Elapsed time 4.29 sec
SLIM_BPR_Recommender: Epoch 151 of 300. Elapsed time 8.79 sec
SLIM_BPR_Recommender: Epoch 226 of 300. Elapsed time 12.78 sec
SLIM_BPR_Recommender: Terminating at epoch 300. Elapsed time 23.41 sec
Deallocating Cython objects
EvaluatorHoldout: Processed 20328 ( 100.00% ) in 12.52 sec. Users per second: 1624
('epoch, batch, topK, lr :300 50 10 0.001', 0.040918383661020505)


In [46]:
# ItemKNN Content Based Filtering
if True:
    from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender 
    itemKNNCBF = ItemKNNCBFRecommender(URM_train, ICM)
    MAP_LIST = []
    tklist = [3]
    shrinklist = [10]

    for tk in tklist:
        for sr in shrinklist:
            itemKNNCBF.fit(shrink=sr, topK = tk, similarity = 'cosine')
            if eval:
                dict_scores = (evaluator_validation.evaluateRecommender(itemKNNCBF))[0][10]
                MAP_LIST.append(('topK, shrink :' + str(tk) + ' ' + str(sr), dict_scores['MAP']))
    if eval:
        sortMap(MAP_LIST)

Similarity column 18495 ( 100 % ), 7147.59 column/sec, elapsed time 0.04 min
EvaluatorHoldout: Processed 20328 ( 100.00% ) in 10.65 sec. Users per second: 1908
('topK, shrink :3 10', 0.013960042392195521)


In [47]:
#%%
if True:
    from MatrixFactorization.PureSVDRecommender import PureSVDRecommender 
    pureSVD = PureSVDRecommender(URM_train)
    MAP_LIST = []
    nfactorlist = [400]

    for n in nfactorlist:
        pureSVD.fit(num_factors=n, random_seed = None)
        if eval:
            dict_scores = (evaluator_validation.evaluateRecommender(pureSVD))[0][10]
            MAP_LIST.append(('num factors :' + str(n) , dict_scores['MAP']))
    if eval:
        print(MAP_LIST)  

PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... Done!
EvaluatorHoldout: Processed 20328 ( 100.00% ) in 16.77 sec. Users per second: 1212
[('num factors :400', 0.024527447339383654)]


In [48]:
# UserKNN Content Based Filtering
if True:
    from KNN.UserKNNCBFRecommender import UserKNNCBFRecommender 
    userKNNCBF = UserKNNCBFRecommender(URM_train, UCM)
    MAP_LIST = []
    tklist = [2500]
    shrinklist = [10]

    for tk in tklist:
        for sr in shrinklist:
            userKNNCBF.fit(shrink=sr, topK = tk, similarity = 'cosine')
            if eval:
                dict_scores = (evaluator_validation.evaluateRecommender(userKNNCBF))[0][10]
                MAP_LIST.append(('topK, shrink :' + str(tk) + ' ' + str(sr), dict_scores['MAP']))
    if eval:
        print(MAP_LIST)
        

Similarity column 30911 ( 100 % ), 1656.55 column/sec, elapsed time 0.31 min
EvaluatorHoldout: Processed 20328 ( 100.00% ) in 20.39 sec. Users per second: 997
[('topK, shrink :2500 10', 0.00865447788326124)]


In [49]:
# LightFM Recommender
if False:
    from RecLib.LFMRec import LFM 
    lfm_rec = LFM(URM_train)
    MAP_LIST = []
    epochsList = [20]
    batchSize = [50]
    tklist = [10]
    lrs = [0.05]
    losses=['bpr']

    for epochsN in epochsList:
        for loss in losses:
            for tk in tklist:
                for lr in lrs:
                    do_not_display_hystory = lfm_rec.fit(epochs=epochsN, no_components=10,
            k=5,
            n=10,
            learning_schedule="adagrad",
            loss=loss,
            learning_rate=lr,
            rho=0.95,
            epsilon=1e-6,
            item_alpha=0.0,
            user_alpha=0.0,
            max_sampled=10,
            random_state=None, num_threads=1,
            verbose=False)
                    if eval:
                        dict_scores = (evaluator_validation.evaluateRecommender(lfm_rec))[0][10]
                        MAP_LIST.append(('epoch, loss, lr :' + str(epochsN) + ' ' + loss + ' '  + str(lr), dict_scores['MAP']))
    if eval:
        sortMap(MAP_LIST)

In [50]:
# UserKNN Collaborative Filtering
if False:
    from KNN.UserKNNCFRecommender import UserKNNCFRecommender 
    userKNNCF = UserKNNCFRecommender(URM_train)
    MAP_LIST = []
    tklist = [600]
    shrinklist = [5]

    for tk in tklist:
        for sr in shrinklist:
            userKNNCF.fit(shrink=sr, topK = tk, similarity = 'cosine')
            if eval:
                dict_scores = (evaluator_validation.evaluateRecommender(userKNNCF))[0][10]
                MAP_LIST.append(('topK, shrink :' + str(tk) + ' ' + str(sr), dict_scores['MAP']))
    if eval:
        sortMap(MAP_LIST)    

In [56]:
# ItemKNN Collaborative Filtering
if True:
    from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
    itemKNN = ItemKNNCFRecommender(URM_train)
    itemKNN.fit(shrink=50, topK=5)

    if eval:
        evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])
        results, _ = evaluator_validation.evaluateRecommender(itemKNN)
        print('MAP: ' + str(results[10]["MAP"]))

Similarity column 18495 ( 100 % ), 7760.86 column/sec, elapsed time 0.04 min
EvaluatorHoldout: Processed 20328 ( 100.00% ) in 10.52 sec. Users per second: 1932
MAP: 0.04484938014762729


In [52]:
if False:
    import xgboost as xgb
    dtest = xgb.DMatrix(X_test, label=y_test)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=3)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)
    
    params = {
    'max_depth': 3,  # the maximum depth of each tree
    'eta': 0.3,  # step for each iteration
    'silent': 1, # keep it quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 3, # the number of classes 
    'eval_metric': 'merror'} # evaluation metric 

    num_round = 20  # the number of training iterations (number of trees)
    model = xgb.train(params,
                  dtrain,
                  num_round,
                  verbose_eval=2,
                  evals=[(dtrain, 'train'), (dval, 'validation')],
                  early_stopping_rounds=20)
    from sklearn.metrics import precision_score

    preds = model.predict(dtest)
    best_preds = np.asarray([np.argmax(line) for line in preds])
    print("Precision: {:.2f} %".format(precision_score(y_test, best_preds, average='macro')))
    

In [53]:
# Graph Based
if True:
    from GraphBased.P3alphaRecommender import P3alphaRecommender
    p3alpha_rec = P3alphaRecommender(URM_train)
    MAP_LIST = []
    similList = [True]
    tklist = [30]
    alphaList = [0.4]

    for simil in similList:
        for tk in tklist:
            for alpha in alphaList:
                do_not_display_hystory = p3alpha_rec.fit(topK=tk, alpha=alpha, normalize_similarity=simil)
                if eval:
                    dict_scores = (evaluator_validation.evaluateRecommender(p3alpha_rec))[0][10]
                    MAP_LIST.append(('Normalize, topK, alpha :' + str(simil)+ ' ' + str(tk) + ' ' + str(alpha), dict_scores['MAP']))
    if eval:
        sortMap(MAP_LIST)
    

EvaluatorHoldout: Processed 20328 ( 100.00% ) in 10.18 sec. Users per second: 1997
('Normalize, topK, alpha :True 30 0.4', 0.046528166488274715)


In [54]:
if True:
    from GraphBased.RP3betaRecommender import RP3betaRecommender
    p3beta_rec = RP3betaRecommender(URM_train)
    MAP_LIST = []
    betaList = [0.1]
    similList = [True]
    tklist = [50]
    alphaList = [0.4]

    for beta in betaList:
        for simil in similList:
            for tk in tklist:
                for alpha in alphaList:
                    do_not_display_hystory = p3beta_rec.fit(topK=tk, alpha=alpha, beta=beta, normalize_similarity=simil)
                    if eval:
                        dict_scores = (evaluator_validation.evaluateRecommender(p3beta_rec))[0][10]
                        MAP_LIST.append(('Normalize, topK, alpha, beta :' + str(simil)+ ' ' + str(tk) + ' ' + str(alpha) + ' ' + str(beta), dict_scores['MAP']))

    if eval:
        sortMap(MAP_LIST)

EvaluatorHoldout: Processed 20328 ( 100.00% ) in 10.35 sec. Users per second: 1964
('Normalize, topK, alpha, beta :True 50 0.4 0.1', 0.04757368003245156)


In [55]:
# Hybrid Recommender
from RecLib.HybridRecommender import *
if True:
    params = [(0.7, 0.1, 0.4, 0.05, 0.5, 1)]
    MAP_LIST = []
    for param in params:
                hybridrecommender = HybridRecommender(URM_train, userKNNCBF, itemKNN, itemKNNCBF, slim_rec, pureSVD, p3alpha_rec, p3beta_rec)
                hybridrecommender.fit(*param)
                if eval:
                    dict_scores = (evaluator_validation.evaluateRecommender(hybridrecommender))[0][10]
                    MAP_LIST.append(('alpha, beta, gamma, delta, epsilon :' + str(param), dict_scores['MAP']))
    if eval:
        sortMap(MAP_LIST)

EvaluatorHoldout: Processed 5001 ( 24.60% ) in 33.38 sec. Users per second: 150
EvaluatorHoldout: Processed 11001 ( 54.12% ) in 1.11 min. Users per second: 165
EvaluatorHoldout: Processed 17001 ( 83.63% ) in 1.62 min. Users per second: 175
EvaluatorHoldout: Processed 20328 ( 100.00% ) in 1.82 min. Users per second: 186
EvaluatorHoldout: Processed 5001 ( 24.60% ) in 30.03 sec. Users per second: 167
EvaluatorHoldout: Processed 11001 ( 54.12% ) in 1.00 min. Users per second: 183
EvaluatorHoldout: Processed 17001 ( 83.63% ) in 1.51 min. Users per second: 188
EvaluatorHoldout: Processed 20328 ( 100.00% ) in 1.85 min. Users per second: 183
EvaluatorHoldout: Processed 5001 ( 24.60% ) in 33.34 sec. Users per second: 150
EvaluatorHoldout: Processed 11001 ( 54.12% ) in 1.10 min. Users per second: 167
EvaluatorHoldout: Processed 17001 ( 83.63% ) in 1.61 min. Users per second: 176
EvaluatorHoldout: Processed 20328 ( 100.00% ) in 1.81 min. Users per second: 187
('alpha, beta, gamma, delta, epsilon 

In [36]:
# Evaluate performance of a recommender against users with 0,1,...,itrMax interactions
rec_to_eval = hybridrecommender
itrMax = 2

if eval:
    evaluateAgainstUsers(rec_to_eval, itrMax, URM_train, URM_test)

In [37]:
# Compare different recommenders
recommendersToCompare = [itemKNN, p3alpha_rec, p3beta_rec, hybridrecommender]

if eval:
    from RecLib.Evaluate import *
    compare(URM_train, URM_test, recommendersToCompare)

In [38]:
# Predict
final_Rec = hybridrecommender

if not eval:
    output = []
    for user_id in target_users:
        output.append((user_id, final_Rec.recommend(user_id, cutoff=10)))

In [39]:
# Writedown results

if not eval:
    import csv
    with open('submission.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["user_id", "item_list"])
        for row in output:
          ranking = ''
          for val in row[1]:
            ranking = ranking + str(val) + ' '
          writer.writerow([row[0], ranking[:-1]])